In [15]:
include("CayleyVerify.jl")
include("DeepPoly.jl")

function dorefa_to_staircase(k::Int)
    n = 2^k - 1
    slopes = zeros(n+1)
    breakpoints = [-Inf]
    for i in 1:n
        push!(breakpoints, (2*i-1)/n - 1)
    end
    push!(breakpoints, Inf)
    
    constant_terms = [-1.0]
    for i in 1:n
        push!(constant_terms, -1.0 + 2*i/n)
    end
    return StaircaseFunction(breakpoints, slopes, constant_terms)
end

dorefa_to_staircase (generic function with 1 method)

In [16]:
using Pickle
using Suppressor
net_from_pickle = Pickle.load(open("./models/MNIST-DoReFa2_Dense256-Dense256.pkl"))
f = dorefa_to_staircase(2)
activation = [f, f]
neural_net = NeuralNetwork(net_from_pickle, activation)
print("net loaded")

net loaded

In [17]:
raw_imgs = Pickle.load(open("./imgs/MNIST_images-for-verification"))
imgs = []
for img in raw_imgs
    img = vcat([w' for w in img] ...)
    img = vcat(img'...)
    push!(imgs, img)
end
labels = Pickle.load(open("./imgs/MNIST_labels-for-verification", "r+"))
labels = Array{Int64}(labels.args[2][5])
labels = [l+1 for l in labels]
print("images loaded")

images loaded

In [18]:
using Printf
upper_bound = 150
lower_bound = 0
count = 1
for (img, label) in zip(imgs, labels)
    @printf("Verify %d-th image \n", count)
    vulnerable = false
    for target_label in 1:10
        if target_label != label
            @suppress begin
                opt_val, opt_sol_x, opt_sol_z = target_attack(neural_net, img, label, target_label, 0.008)
                if opt_val > 0
                    vulnerable = true
                    adv_img = [opt_sol_x[1, j] for j in 1:784]
                    pred = predict(neural_net, adv_img)
                    if pred != label
                        upper_bound -= 1
                        break
                    end
                end
            end
        end
    end
    if vulnerable == false
        lower_bound += 1
    end
    count += 1
end

Verify 1-th image 
Verify 2-th image 
Verify 3-th image 
Verify 4-th image 
Verify 5-th image 
Verify 6-th image 
Verify 7-th image 
Verify 8-th image 
Verify 9-th image 
Verify 10-th image 
Verify 11-th image 
Verify 12-th image 
Verify 13-th image 
Verify 14-th image 
Verify 15-th image 
Verify 16-th image 
Verify 17-th image 
Verify 18-th image 
Verify 19-th image 


UndefVarError: UndefVarError: `predict` not defined

In [19]:
start = time_ns()
1+1
finish = time_ns()
(finish - start)/1e09

0.000349416

In [20]:
start

0x000004c7335c1a08